**Load Data and do preprocessing**

***I have mounted my drive and loaded our training dataset in a DataFrame and in the csv file i have only used columns hindi and english and left the index column that was present in the csv.***

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
dataset=pd.read_csv('/content/drive/MyDrive/train.csv',encoding='utf-8',usecols=['hindi','english'])
#dataset.head()

**Pre-requisite for hindi text processing**

***Cloning the indic_nlp_library and setting the path . indic_nlp library will be used pre-processing the hindi text provided in training dataset.***

In [ ]:
!git clone "https://github.com/anoopkunchukuttan/indic_nlp_library"
!git clone https://github.com/anoopkunchukuttan/indic_nlp_resources.git
!pip install Morfessor
# The path to the local git repo for Indic NLP library
INDIC_NLP_LIB_HOME=r"/content/indic_nlp_library"

# The path to the local git repo for Indic NLP Resources
INDIC_NLP_RESOURCES="/content/indic_nlp_resources"
import sys
sys.path.append(r'{}'.format(INDIC_NLP_LIB_HOME))
from indicnlp import common
common.set_resources_path(INDIC_NLP_RESOURCES)
from indicnlp import loader
loader.load()

Cloning into 'indic_nlp_library'...
remote: Enumerating objects: 1271, done.
remote: Counting objects: 100% (93/93), done.
remote: Compressing objects: 100% (68/68), done.
remote: Total 1271 (delta 50), reused 54 (delta 25), pack-reused 1178
Receiving objects: 100% (1271/1271), 9.56 MiB | 10.98 MiB/s, done.
Resolving deltas: 100% (654/654), done.
Cloning into 'indic_nlp_resources'...
remote: Enumerating objects: 133, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 133 (delta 0), reused 2 (delta 0), pack-reused 126
Receiving objects: 100% (133/133), 149.77 MiB | 28.20 MiB/s, done.
Resolving deltas: 100% (51/51), done.
Checking out files: 100% (28/28), done.


***Importing libraries for pre-processing Hindi and English Text***

In [ ]:
import re
import string
import random
from unicodedata import normalize
from indicnlp.normalize.indic_normalize import IndicNormalizerFactory
from indicnlp.tokenize import indic_tokenize



***Here I am building my vocabulary where SOS_token is the start token , EOS_token is the end token , UNK_token is for unknown token, PAD_token is for padding.***

In [ ]:
SOS_token = '<start>' #start token
EOS_token = '<end>'   #end token
UNK_token = '<unk>'   #unknown token
PAD_token = '<pad>'   #pad token

SOS_idx = 0  #index for start token
EOS_idx = 1  #index for end token
UNK_idx = 2  #index for unknown token
PAD_idx = 3  #index for pad token

class Lang:
    def __init__(self, name):
        # name denotes the name of the language sent as input i.e. Hindi or English
        self.name = name  

        #word2count is used to count the times a word appreas in the vocabulary
        self.word2count = {}

        # n_words denotes the number of unique words in constructed vocabulary
        self.n_words = 4  # Count SOS,EOS,PAD,UNK

        #index2word provides the word from the vocabulary we will build given  the index
        self.index2word = {
            SOS_idx: SOS_token,
            EOS_idx: EOS_token,
            UNK_idx: UNK_token,
            PAD_idx: PAD_token
        }

        #word2index provides the index from the vocabulary we will build given  the word
        self.word2index = {v: k for k, v in self.index2word.items()}
   
    #given a sentence I am splitting it in words and adding it to our vocabulary
    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            # since my index starts from 0 and I have initialised n_words to 4 and intitially my vocab contains just SOS,EOS,PAD,UNK , I am putting the n_words count as index of the current word encountered
            self.word2index[word] = self.n_words 
             
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

***Here I have defined functions for cleaning the source(hindi) text and target(english) text***

In [ ]:
#clean_text function takes in the text and it removes puntuation marks, numbers,quotes from the provided text and returns the cleaned text
def clean_text(text):
    text = text.replace(u',','')
    text = text.replace(u'"','')
    text = text.replace(u'"','')
    text = text.replace(u"‘‘",'')
    text = text.replace(u"’’",'')
    text = text.replace(u"''",'')
    text = text.replace(u"।",'')
    text=text.replace(u',','')
    text=text.replace(u'"','')
    text=text.replace(u'(','')
    text=text.replace(u')','')
    text=text.replace(u'"','')
    text=text.replace(u':','')
    text=text.replace(u"'",'')
    text=text.replace(u"‘‘",'')
    text=text.replace(u"’’",'')
    text=text.replace(u"''",'')
    text=text.replace(u".",'')
    text=text.replace(u"-",'')
    text=text.replace(u"।",'')
    text=text.replace(u"?",'')
    text=text.replace(u"\\",'')
    text=text.replace(u"_",'')
    text=text.replace("'", "")
    text=text.replace('"', "")
    text= re.sub("'", '', text)
    text= re.sub("’", '', text)
    text=re.sub('[0-9+\-*/.%]', '', text)
    text=text.strip()
    text=re.sub(' +', ' ',text)
    exclude = set(string.punctuation)
    text= ''.join(ch for ch in text if ch not in exclude)
    return text

#this function is used to pre-process english sentences
def pre_process_english_sentence(line):
    #lower casing the text
    line = line.lower()
    #cleaning the text 
    line = clean_text(line)
    re_print = re.compile('[^%s]' % re.escape(string.printable))
    #I first normalise the sentences using unicode normalise to NDF(cannonical decomposition) and then convert to ascii ignoring the errors
    line = normalize('NFD', line).encode('ascii', 'ignore') 
    line = line.decode('UTF-8')
    line = line.split()
    line = [re_print.sub('', w) for w in line]
    line = [word for word in line if word.isalpha()]
    line = ' '.join(line)
    return line

#this function is used to pre-process hindi sentences
def pre_process_hindi_sentence(line):
    #cleaning the text
    line = clean_text(line)
    factory = IndicNormalizerFactory()
    #remove_nuktas is set to false as it usually removes things like vishargya from the text and I did not need that to be done
    normalizer = factory.get_normalizer("hi",remove_nuktas=False)
    line = normalizer.normalize(line)
    tokens = list()
    #tokenising the text using trial_tokenize of indic library
    for t in indic_tokenize.trivial_tokenize(line):
        tokens.append(t)
    line = tokens
    line = [word for word in line if not re.search(r'\d', word)]
    line = ' '.join(line)
    return (line)

#this functions appends the preprocessed data to a list of list 
def to_pairs(df):
    english_lines =df['english'].tolist()
    hindi_lines = df['hindi'].tolist()
    pairs = []
    for i in range(len(hindi_lines)):
        pairs.append([])
        pairs[i].append(pre_process_hindi_sentence(hindi_lines[i]))
        pairs[i].append(pre_process_english_sentence(english_lines[i]))
    return pairs

#this preparesmy vocabulary for hindi and english language by using the words in the training dataset
def prepareData(pairs):
    input_lang = Lang('hin')
    output_lang = Lang('eng')
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    return input_lang, output_lang


pairs_all = to_pairs(dataset) #pairs_all is a list of list containing processed hindi and its corresponding english text
input_lang, output_lang = prepareData(pairs_all) #input_lang and output_lang are objects of the Lang class defined earlier which contains information about the vocabulary

***Let us check if the pairs_all has been constructed correctly***

In [ ]:
for i,j in enumerate(pairs_all):
  print(i,j)
  break

0 ['एल सालवाडोर मे जिन दोनो पक्षों ने सिविलयुद्ध से वापसी ली उन्होंने वही काम किये जो कैदियों की कश्मकश के निदान हैं', 'in el salvador both sides that withdrew from their civil war took moves that had been proven to mirror a prisoners dilemma strategy']


***Let us now split the pairs all into two lists one containing all source sentences and the other containing the target sentences . This will be helpful while constructing the train and the validation dataset.***

In [ ]:
source_to_target = {}
i=0
for index,source_target in enumerate(pairs_all):
    source=source_target[0]
    target=source_target[1]
    if source in source_to_target:
        source_to_target[source].append(target)
    else:
        source_to_target[source] = [target]
#print(source_to_target)
source_sents, target_sents = zip(*source_to_target.items())
#print(source_sents[0])
#print(target_sents[0])

***Checking if my source_sents are correctly added***

In [ ]:
for i in source_sents:
  print(i)
  break

एल सालवाडोर मे जिन दोनो पक्षों ने सिविलयुद्ध से वापसी ली उन्होंने वही काम किये जो कैदियों की कश्मकश के निदान हैं


**Splitting into train,val data (90%,10%) as test set was provided every week**





In [ ]:
import numpy as np
#used random_seed to ensure that my every split is uniform
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

source_length = len(source_sents)
inidices = np.random.permutation(source_length)

#spliting the dataset in the ratio of 9:1  (train:validation ratio)
training_indices = inidices[:int(source_length*0.9)] 
dev_indices = inidices[int(source_length*0.9):]

#preparing the training dataset
training_source = [source_sents[i] for i in training_indices]
dev_source = [source_sents[i] for i in dev_indices]

training_target = [target_sents[i] for i in training_indices]
dev_target = [target_sents[i] for i in dev_indices]

# Unwrap training examples
training_t = []
training_s = []
for source, tt in zip(training_source, training_target):
    for target in tt:
        training_t.append(target)
        training_s.append(source)

training_source = training_s
training_target = training_t

***Checking the number of words in my hindi vocabulary (constructed from the training dataset)***

In [ ]:
input_lang.n_words

46185

***Checking the number of words in my english vocabulary(constructed from training dataset)***

In [ ]:
output_lang.n_words

32596

In [ ]:
import numpy as np

***checking the maximum length of the source and target sentence so that it is helpful while padding the shorter sentences , to make all of same length and then convert to a tensor object.***

In [ ]:
df = pd.DataFrame(pairs_all)
df.columns = ["hindi", "english"]
lines = df
lenght_list=[]
for l in lines.hindi:
    lenght_list.append(len(l.split(' ')))
max_length_src = np.max(lenght_list)
print (max_length_src)
lenght_list=[]
for l in lines.english:
    lenght_list.append(len(l.split(' ')))
max_length_tar = np.max(lenght_list)
print (max_length_tar)
df.head()

394
301


,hindi,english
0,एल सालवाडोर मे जिन दोनो पक्षों ने सिविलयुद्ध स...,in el salvador both sides that withdrew from t...
1,मैं उनके साथ कोई लेना देना नहीं है,i have nothing to do with them
2,हटाओ रिक,fuck them rick
3,क्योंकि यह एक खुशियों भरी फ़िल्म है,because its a happy film
4,The thought reaching the eyes,the thought reaching the eyes


**Starting the Model**

***Importing the necessary libraries for our model***

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.nn.init as init
#setting the device to cuda if its available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


**Encoder**

In [ ]:
class EncoderRNN(nn.Module):
    def __init__(self, vocab_size, hidden_size, n_layers=1):
        super(EncoderRNN, self).__init__()

        self.vocab_size = vocab_size
        self.hidden_size = hidden_size
        self.n_layers = n_layers

        self.embedding = nn.Embedding(vocab_size, hidden_size)
        init.normal_(self.embedding.weight, 0.0, 0.2)

        self.lstm = nn.LSTM(
            hidden_size,
            int(hidden_size/2),  # Bi-directional processing will ouput vectors of double size, therefore I reduced output dimensionality
            num_layers=n_layers,
            batch_first=True,  # First dimension of input tensor will be treated as a batch dimension
            bidirectional=True
        )

    # word_inputs: (batch_size, seq_length), h: (h_or_c, layer_n_direction, batch, seq_length)
    def forward(self, word_inputs, hidden):         
        # embedded (batch_size, seq_length, hidden_size)
        embedded = self.embedding(word_inputs)
        # output (batch_size, seq_length, hidden_size*directions)
        # hidden (h: (num_layers*directions, batch_size, hidden_size),
        #         c: (num_layers*directions, batch_size, hidden_size))
        output, hidden = self.lstm(embedded, hidden)
        return output, hidden

    def init_hidden(self, batches):
        h_s = torch.zeros(self.n_layers*2, batches, int(self.hidden_size/2)).contiguous().to(device)
        c_s = torch.zeros(self.n_layers*2, batches, int(self.hidden_size/2)).contiguous().to(device)
        hidden = (h_s, c_s)
        return hidden

***I have just tested my encoded with a small dummy example to make sure it is working properly***

In [ ]:
vocab_size = 10
hidden_dim = 10
n_layers = 2

encoder_test = EncoderRNN(vocab_size, hidden_dim, n_layers).to(device)
print(encoder_test)

# Recurrent network requires initial hidden state
encoder_hidden = encoder_test.init_hidden(1)

# Test input of size (1x3), one sequence of size 3
word_input = torch.LongTensor([[1, 2, 3]]).to(device)
#print(word_input.shape," ",encoder_hidden.shape)

encoder_outputs, encoder_hidden = encoder_test(word_input, encoder_hidden)

# encoder_outputs: (batch_size, seq_length, hidden_size)
# encoder_hidden[0, 1]: (n_layers*2, batch_size, hidden_size/2)
print(encoder_outputs.shape, encoder_hidden[0].shape, encoder_hidden[1].shape)

EncoderRNN(
  (embedding): Embedding(10, 10)
  (lstm): LSTM(10, 5, num_layers=2, batch_first=True, bidirectional=True)
)
torch.Size([1, 3, 10]) torch.Size([4, 1, 5]) torch.Size([4, 1, 5])


**Decoder**

In [ ]:
class DecoderRNN(nn.Module):
    def __init__(self, vocab_size, hidden_size, n_layers=1):
        super(DecoderRNN, self).__init__()

        self.vocab_size = vocab_size
        self.hidden_size = hidden_size
        self.n_layers = n_layers

        self.embedding = nn.Embedding(vocab_size, hidden_size)
        init.normal_(self.embedding.weight, 0.0, 0.2)

        self.lstm = nn.LSTM(hidden_size, hidden_size, num_layers=n_layers, batch_first=True, bidirectional=False) # I have used unidirectional LSTM for decoder 

    def forward(self, word_inputs, hidden):
        # we run this one by one
        # embedded (batch_size, 1, hidden_size)
        embedded = self.embedding(word_inputs.to(device)).unsqueeze_(1)
        output, hidden = self.lstm(embedded, hidden)
        return output, hidden

***I did a small test for the decoder code as well to make sure it works properly.***

In [ ]:
decoder_test = DecoderRNN(vocab_size, hidden_dim, n_layers).to(device)
print(decoder_test)

word_inputs = torch.LongTensor([[1, 2, 3]]).to(device)

decoder_hidden_h = encoder_hidden[0].permute(1, 0, 2).reshape(1, 2, 10).permute(1, 0, 2)
decoder_hidden_c = encoder_hidden[1].permute(1, 0, 2).reshape(1, 2, 10).permute(1, 0, 2)


for i in range(3):
    input = word_inputs[:, i]
    decoder_output, decoder_hidden = decoder_test(input, (decoder_hidden_h, decoder_hidden_c))
    decoder_hidden_h, decoder_hidden_c = decoder_hidden
    print(decoder_output.size(), decoder_hidden_h.size(), decoder_hidden_c.size())

DecoderRNN(
  (embedding): Embedding(10, 10)
  (lstm): LSTM(10, 10, num_layers=2, batch_first=True)
)
torch.Size([1, 1, 10]) torch.Size([2, 1, 10]) torch.Size([2, 1, 10])
torch.Size([1, 1, 10]) torch.Size([2, 1, 10]) torch.Size([2, 1, 10])
torch.Size([1, 1, 10]) torch.Size([2, 1, 10]) torch.Size([2, 1, 10])


**Model**

***Putting the encoder decoder structure to build the seq2seq model***

In [ ]:
class Seq2seq(nn.Module):
    def __init__(self, input_vocab_size, output_vocab_size, hidden_size, n_layers,device):
        super(Seq2seq, self).__init__()

        self.n_layers = n_layers
        self.hidden_size = hidden_size
        self.device = device
        #EncoderRNN takes in the input(hindi) language vocabulary size , hidden dimension and also the number of layers
        #I did to(self.device) to ensure the input and the hidden layer outputs are in the same device while running on GPUs
        self.encoder = EncoderRNN(input_vocab_size, hidden_size, self.n_layers).to(self.device)

        #DecoderRNN takes in the output(english) language vocabulary size , hidden dimension and also the number of layers
        #I did to(self.device) to ensure the input and the hidden layer outputs are in the same device while running on GPUs
        self.decoder = DecoderRNN(output_vocab_size, hidden_size, self.n_layers).to(self.device)

        self.W = nn.Linear(hidden_size, output_vocab_size)
        init.normal_(self.W.weight, 0.0, 0.2) #initialise the weights

        self.softmax = nn.Softmax()
        
    def _forward_encoder(self, x):
        # x is (batch size, sequence length)
        batch_size = x.shape[0]
        #initialising the hidden inputs according to the batch_size
        init_hidden = self.encoder.init_hidden(batch_size)
        #send the input x and use .to(device) on x to ensure it is in the same device as init_hidden while using GPUs
        encoder_outputs, encoder_hidden = self.encoder(x.to(device), init_hidden)
        encoder_hidden_h, encoder_hidden_c = encoder_hidden

        self.decoder_hidden_h = encoder_hidden_h.contiguous().permute(1,0,2).reshape(batch_size, self.n_layers, self.hidden_size).permute(1,0,2)
        self.decoder_hidden_c = encoder_hidden_c.contiguous().permute(1,0,2).reshape(batch_size, self.n_layers, self.hidden_size).permute(1,0,2)
        return self.decoder_hidden_h, self.decoder_hidden_c

    def forward_train(self, x, y):
        decoder_hidden_h, decoder_hidden_c = self._forward_encoder(x)
        #print(decoder_hidden_h.is_contiguous," ",decoder_hidden_c.is_contiguous)
        H = []
        for i in range(y.shape[1]):
            input = y[:, i]
            decoder_output, decoder_hidden = self.decoder(input, (decoder_hidden_h.contiguous(), decoder_hidden_c.contiguous()))
            decoder_hidden_h, decoder_hidden_c = decoder_hidden
            # h: (batch_size, vocab_size)
            h = self.W(decoder_output.squeeze(1))
            # h: (batch_size, vocab_size, 1)
            H.append(h.unsqueeze(2))

        # H: (batch_size, vocab_size, seq_len)
        return torch.cat(H, dim=2)

    #this is used during test time
    def forward(self, x):
        decoder_hidden_h, decoder_hidden_c = self._forward_encoder(x)

        current_y = SOS_idx
        result = [current_y]
        counter = 0
        #I have set the limit of counter to be 100 as i did not want my predicted sentences to be too long
        while current_y != EOS_idx and counter < 100:
            input = torch.tensor([current_y])
            decoder_output, decoder_hidden = self.decoder(input, (decoder_hidden_h, decoder_hidden_c))
            decoder_hidden_h, decoder_hidden_c = decoder_hidden
            # h: (vocab_size)
            h = self.W(decoder_output.squeeze(1)).squeeze(0)
            y = self.softmax(h)
            _, current_y = torch.max(y, dim=0)
            current_y = current_y.item()
            result.append(current_y)
            counter += 1

        return result

**We have prepared input output pairs which are strings but input to our model should be tensors. So need to convert to tensors**

In [ ]:
import os 
data_dir='/content/drive/MyDrive/'

In [ ]:
#this function takes in the lang object(which denotes the either hindi vocab set or english vocab set) and the sentence and converts each word to its index
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]

#this function takes the lang object(which denotes the either hindi vocab set or english vocab set) and the sentence and also the type(which is either hindi or english)
def tensorFromSentence(lang, sentence,type):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_idx) #appended the end of sentence token at the end of the sentence
    indexes.insert(0, SOS_idx) #appended the start of sentence token at the start of the sentence
    if type=='hindi':
      max_seq_length=2+max_length_src #if the type is hindi the max length is set to 2+the max length of hindi sentences encountered in the training dataset(2 because we appended 2 tokens)
    else:
      max_seq_length=2+max_length_tar  #if the type is english the max length is set to 2+the max length of english sentences encountered in the training dataset(2 because we appended 2 tokens)
    if len(indexes) < max_seq_length:
        indexes += [PAD_idx] * (max_seq_length - len(indexes)) # padding shorter sentences with the pad token index
    return torch.tensor(indexes, dtype=torch.long, device=device) # converting the indexes to a tensor object


def tensorsFromPair(source,target):
    #I unsqeezed the tensors as I wanted the size to be (sequence length) and not (1,sequence length)
    input_tensor = tensorFromSentence(input_lang, source,'hindi').unsqueeze(1)
    target_tensor = tensorFromSentence(output_lang, target,'english').unsqueeze(1)
    return (input_tensor, target_tensor)


#constructing training and validation set tensors for source(hindi) and target(english) language
training = []
for source_sent, target_sent in zip(training_source, training_target):
    #print(source_sent)
    training.append(tensorsFromPair(source_sent, target_sent))

x_training, y_training = zip(*training)
x_training = torch.transpose(torch.cat(x_training, dim=-1), 1, 0)
y_training = torch.transpose(torch.cat(y_training, dim=-1), 1, 0)
torch.save(x_training, os.path.join(data_dir, 'x_training.bin'))
torch.save(y_training, os.path.join(data_dir, 'y_training.bin'))

x_development = []
for source_sent in dev_source:
    tensor = tensorFromSentence(input_lang, source_sent, 'hindi').unsqueeze(1)
    x_development.append(tensor)

x_development = torch.transpose(torch.cat(x_development, dim=-1), 1, 0)
torch.save(x_development, os.path.join(data_dir, 'x_development.bin'))
'''
x_test = []
for source_sent in test_source:
    tensor = tensorFromSentence(input_lang, source_sent, 'hindi').unsqueeze(1)
    x_test.append(tensor)

x_test = torch.transpose(torch.cat(x_test, dim=-1), 1, 0)
torch.save(x_test, os.path.join(data_dir, 'x_test.bin'))
'''

"\nx_test = []\nfor source_sent in test_source:\n    tensor = tensorFromSentence(input_lang, source_sent, 'hindi').unsqueeze(1)\n    x_test.append(tensor)\n\nx_test = torch.transpose(torch.cat(x_test, dim=-1), 1, 0)\ntorch.save(x_test, os.path.join(data_dir, 'x_test.bin'))\n"

***The unindex_words function will be used during the test time when the decoder provides us the indices of the predicted words we translate the indices back to words***

In [ ]:
def unindex_words(lang, indices):
        return [lang.index2word[i] for i in indices]

**Training our model**

In [ ]:
from torch.optim import Adam
#initialising our seq2seq model with no. of layers as 1 and hidden dimension as 256
model = Seq2seq(input_lang.n_words, output_lang.n_words, 256, 1,device).to(device)
#I have used Adam as my optimiser and have set my initial learning rate to 0.0001
optim = Adam(model.parameters(), lr=0.0001)

In [ ]:
#checking specifications of my model
model

Seq2seq(
  (encoder): EncoderRNN(
    (embedding): Embedding(46185, 256)
    (lstm): LSTM(256, 128, batch_first=True, bidirectional=True)
  )
  (decoder): DecoderRNN(
    (embedding): Embedding(32596, 256)
    (lstm): LSTM(256, 256, batch_first=True)
  )
  (W): Linear(in_features=256, out_features=32596, bias=True)
  (softmax): Softmax(dim=None)
)

**We will train our model in batches**

In [ ]:
import math
#since I wanted to train my model in batches the batch_generator function takes in the size of the input to be batched and the batch size
#the function yields the set of indices required for the current batch
def batch_generator(batch_indices, batch_size):
    batches = math.ceil(len(batch_indices)/batch_size)
    for i in range(batches):
        batch_start = i*batch_size
        batch_end = (i+1)*batch_size
        if batch_end > len(batch_indices):
            yield batch_indices[batch_start:]
        else:
            yield batch_indices[batch_start:batch_end]

In [ ]:
# for calculating the loss I have used the CrossEntropyLoss function
cross_entropy = nn.CrossEntropyLoss()


In [ ]:
pip install nltk

***I have used corpus_bleu as a scoring metric to check my models performance on validation set(constructed from the provided training dataset) across different epochs***

In [ ]:
from nltk.translate.bleu_score import corpus_bleu

def bleu(n):
    weights = [1.0/n]*n + [0.0]*(4-n)
    return lambda list_of_references, list_of_hypothesis: corpus_bleu(list_of_references, list_of_hypothesis, weights)

def accuracy(list_of_references, list_of_hypothesis):
    total = 0.0
    for references, hypothesis in zip(list_of_references, list_of_hypothesis):
        total += 1.0 if tuple(hypothesis) in set(references) else 0.0
    return total / len(list_of_references)

score_functions = {'BLEU-{}'.format(i):bleu(i) for i in range(1, 5)}
score_functions['Accuracy'] = accuracy

def score(model, X, target, desc='Scoring...'):
    scores = {name:0.0 for name in score_functions.keys()}
    length = len(target)
    list_of_hypothesis = []
    for i, x in tqdm(enumerate(X),
                     desc=desc,
                     total=length):
        y = model(x.unsqueeze(0).to(device))
        hypothesis = unindex_words(output_lang,y[1:-1])  # Remove SOS and EOS from y
        list_of_hypothesis.append(hypothesis)

    for name, func in score_functions.items():
        score = func(target, list_of_hypothesis)
        scores[name] = score

    return scores

***I have used tqdm_notebook just to monitor the progress***

In [ ]:
from tqdm import tqdm_notebook as tqdm

BATCH_SIZE = 16 # i have taken batch size to be 16
total_batches = int(len(x_training)/BATCH_SIZE) + 1  # total batches denote the numer of batches of size 16 
indices = list(range(len(x_training))) # it takes the length of the training set

early_stop_after = 3 # early stop counter is 3 so that we can do a early stop if we see no improvement for 3 back to back epochs
early_stop_counter = 0
best_model = None

best_score = 0.0
scoring_metric = 'BLEU-1'
scores_history = []
loss_history = []

for epoch in range(5):
    # Training
    total_loss = 0.0
    '''
    this part to be used only when we have a model state to start with
    if epoch==0:
      model.load_state_dict(torch.load('/content/drive/MyDrive/model_lstm_9_april.pt'))
    '''
    for step, batch in tqdm(enumerate(batch_generator(indices, BATCH_SIZE)),
                              desc='Training epoch {}'.format(epoch+1),
                              total=total_batches):
          x = x_training[batch, :].to(device)
          # y for teacher forcing is all sequence without a last element
          y_tf = y_training[batch, :-1].to(device)
          # y for loss calculation is all sequence without a last element
          y_true = y_training[batch, 1:].to(device)
          # (batch_size, vocab_size, seq_length)
          H = model.forward_train(x, y_tf)
          loss = cross_entropy(H, y_true)

          assert loss.item() > 0 # made sure that the loss is always >0 

          optim.zero_grad()
          loss.backward()
          optim.step()

          total_loss += loss.item()

    loss_history.append(total_loss/total_batches)
    torch.save(model.state_dict(), '/content/drive/MyDrive/model_lstm_9_april.pt')
    print('Epoch {} training is finished, loss: {:.4f}'.format(epoch+1, total_loss/total_batches))

    desc = 'Validating epoch {}'.format(epoch+1)
    scores = score(model, x_development.to(device), dev_target, desc=desc)
    scores_str = '\n'.join(['{}: {:.4f}'.format(name, score) for name, score in scores.items()])
    scores_history.append(scores)

    print ('Epoch {} validation is finished.\n{}'.format(
        epoch+1, scores_str
    ))

    metric = scores[scoring_metric]
    
    # Early Stop
    if metric > best_score:
        early_stop_counter = 0
        print('The best model is found, resetting early stop counter.')
        best_score = metric
        best_model = model
        torch.save(best_model.state_dict(), '/content/drive/MyDrive/model_lstm_best_9_april_new.pt')
    else:
        early_stop_counter += 1
        print('No improvements for {} epochs.'.format(early_stop_counter))
        if early_stop_counter >= early_stop_after:
            print('Early stop!')
            break

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



Epoch 1 training is finished, loss: 0.1475


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


...


Epoch 3 training is finished, loss: 0.1376


/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


Epoch 3 validation is finished.
BLEU-1: 0.0017
BLEU-2: 0.0073
BLEU-3: 0.0118
BLEU-4: 0.0151
Accuracy: 0.0000
The best model is found, resetting early stop counter.



Epoch 4 training is finished, loss: 0.1333



Epoch 4 validation is finished.
BLEU-1: 0.0017
BLEU-2: 0.0074
BLEU-3: 0.0119
BLEU-4: 0.0152
Accuracy: 0.0000
The best model is found, resetting early stop counter.


**Evaluation**

In [ ]:
#checking the model's performance on the dev set prepared from the training dataset provided
examples = zip(dev_source[10:20], dev_target[10:20], x_development[10:20]) #checked for only 10 sentences
for source, target, x in examples:
    y = model(x.unsqueeze(0))
    translation = ' '.join(unindex_words(output_lang,y[1:-1]))
    source = ' '.join(source)
    references = '\n'.join([' '.join(t) for t in target])

    print('Source: "{}"\nReferences:\n{}\nTranslation: "{}"\n'.format(source, references, translation))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Source: "य ा   फ ़ ि र   म ै ं   ज ा   क र   उ न ् ह े ं   ब त ा त ा   ह ू ँ   द े ख ो   य े   क र न े   क ी   ज ़ ् र ू र त   ह ै"
References:
c o m e   a n d   t e l l   m e   w h a t   i t   i s
Translation: "or tell me what you do is so much more"

Source: "म ै ं   द े त ी   ह ू ं   क ु छ"
References:
i   c a n   l e t   y o u   h a v e   s o m e
Translation: "i love something"

Source: "य ा र   त ु म   स च   ह ै   क ि   भ ा व न ा   क ी   ज ा ँ च   क र न े   म ि ल ा   य ह   ह ै   क ि   ह म   क ् य ा   न ह ी ं   ह ै"
References:
o n   s o m e   t y p e   s h i t   t h i s   i s   n o t   w h a t   w e   d o
Translation: "the man is that you dont have to say that we have to do that"

Source: "व ह   प श ् च ि म   अ फ ् र ी क ा   क े   ब ा ह र   प ै स े   क ी   ए क   ट न   ध ो ख ा"
References:
h e   s w i n d l e d   a   t o n   o f   m o n e y   o u t   o f   w e s t   a f r i c a
Translation: "hes the money in the middle of the middle of the east"

Source: "स ो क ् र े ट ् स   क ी   

**Using the Dev set provided**

In [ ]:
#loading the dev set provided for week 2
hindi_statements=pd.read_csv('/content/drive/MyDrive/testhindistatements.csv',encoding='utf-8',usecols=['hindi'])

In [ ]:
#converting dataframe object to a list
test_week2=hindi_statements.values.tolist()

In [ ]:
#preprocessing the sentences and storing it in a list
test_week2_preprocessed=[]
for i in range(len(test_week2)):
    clean_line=pre_process_hindi_sentence(str(test_week2[i]))
    test_week2_preprocessed.append(clean_line)

In [ ]:
#checking one of the preprocessed text
test_week2_preprocessed[0]

'तालियां अब इसने मेरे मन को उड़ा दिया'

In [ ]:
def indexesFromSentence_test(lang, sentence):
    list_index=[]
    for word in sentence.split(' '):
      try:
        list_index.append(lang.word2index[word])
      except:
        #if a word is not present in the vocabulary made from training set I defined it as unknown token
        list_index.append(lang.word2index[UNK_token])
    return list_index


def tensorFromSentence_test(lang, sentence,type):
    indexes = indexesFromSentence_test(lang, sentence)
    #print(len(indexes))
    indexes.append(EOS_idx)
    indexes.insert(0, SOS_idx)
    if type=='hindi':
      max_seq_length=2+max_length_src
    else:
      max_seq_length=2+max_length_tar
    if len(indexes) < max_seq_length:
        indexes += [PAD_idx] * (max_seq_length - len(indexes))
    return torch.tensor(indexes, dtype=torch.long, device=device)

In [ ]:
#converting the list to a tensor object so that we can send it as input to our model
test_week2_tensor=[]
for source_sent in test_week2_preprocessed:
    tensor = tensorFromSentence_test(input_lang, source_sent, 'hindi').unsqueeze(1)
    test_week2_tensor.append(tensor)
test_week2_tensor = torch.transpose(torch.cat(test_week2_tensor, dim=-1), 1, 0)

In [ ]:
#checking the shape of the tensor 
test_week2_tensor.shape

torch.Size([24102, 396])

In [ ]:
#initialising the model
model = Seq2seq(input_lang.n_words, output_lang.n_words, 256, 1,device).to(device)
#loading the saved state of the model
model.load_state_dict(torch.load('/content/drive/MyDrive/model_lstm_best_9_april_new.pt'))
examples = zip(test_week2_preprocessed,test_week2_tensor)
file1 = open("/content/drive/MyDrive/answer_27.txt","w") #writing the predicted outputs to a file 
for source,  x in examples:
    y = model(x.unsqueeze(0).to(device))
    translation = ' '.join(unindex_words(output_lang,y[1:-1]))
    source = ' '.join(source)
    
    file1.write(translation+'\n')
    print('Source: "{}"\nTranslation: "{}"\n'.format(source,  translation)) # I printed the source and model's predicted translations here
file1.close()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:64: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Streaming output truncated to the last 5000 lines.
Translation: "look here you pelle"

Source: "क ् य ो ं   ह म ा र े   स म य   क े   स ब स े   ब ु र े   य ु द ् ध   क े   प ी ड ़ ि त   स ी र ि य ा ई   श र ण ा र ् थ ि य ो ं   क े   ल ि ए   क ो ई   व ि श ा ल   प ु न र ् व ा स   क ा र ् य क ् र म   न ह ी ं   ह ै"
Translation: "why we have to be the most of the most of the government of the system of the government of the american system"

Source: "क ो ई   ब ह ु त   अ च ् छ ा"
Translation: "no good"

Source: "य ह   उ स न े   ऐ स ा   क ् य ो ं   क ि य ा   व ह   य ह   ह ै   क ् य ा   न ह ी ं   ह ै"
Translation: "who does he do it because he doesnt know"

Source: "व ह   म ु ख ् य   र स ो इ य ा   ह ै"
Translation: "hes the king of the river"

Source: "म ै ं   अ प न ी   म ा ं   क े   स ा थ   र ह न ा   च ा ह त ा   ह ू ं"
Translation: "i want to my mother with your own"

Source: "त ु म ् ह ा र े   स ा थ   र ह क र"
Translation: "you with your neck"

Source: "व ै न   म े ं   र ह ो   स च   म े ं"
T